In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from fuzzywuzzy import fuzz
import math
from shapely.geometry import Point
from geopy.distance import distance

## Step 1: Read-In Data

In [2]:
fileString_vz = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/01_prep/03_FareZones/vvs_haltestellenverzeichnis_v2.csv"
fileString_gtfs = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/01_prep/03_FareZones/vvs_stops.csv"
fileString_vvs_area_geom = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/01_prep/03_FareZones/vvs_area_geom.shp"

In [3]:
df_vz = pd.read_csv(fileString_vz)

In [4]:
df_vz.head(3)

,ort_1,ort_2,haltestelle,nr,tarifzone
0,Affalterbach,NaN,Birkhau,3600,3
1,Affalterbach,NaN,Klingenstraße,3601,3
2,Affalterbach,NaN,Marbacher Straße,3599,3


In [5]:
df_vz.tail(3)

,ort_1,ort_2,haltestelle,nr,tarifzone
3802,Wüstenrot,NaN,Stangenbach,32449,6
3803,Wüstenrot,NaN,Stangenbach Haus Waldesruh,32453,6
3804,Wüstenrot,NaN,Vorderbüchelberg Abzweig,32320,6


In [6]:
df_gtfs = pd.read_csv(fileString_gtfs)

In [7]:
gdf_gtfs = gpd.GeoDataFrame(df_gtfs,
                            geometry = gpd.points_from_xy(df_gtfs.stop_lon, df_gtfs.stop_lat),
                           )
gdf_gtfs = gdf_gtfs.set_crs(epsg=4326)

In [8]:
gdf_gtfs.head(3)

,stop_id,stop_name,stop_lat,stop_lon,geometry
0,de:08111:100:1:1,Stammheim,48.850046,9.156174,POINT (9.15617 48.85005)
1,de:08111:100:2:3,Stammheim,48.849443,9.156022,POINT (9.15602 48.84944)
2,de:08111:100:2:4,Stammheim,48.850217,9.156093,POINT (9.15609 48.85022)


In [9]:
gdf_gtfs.tail(3)

,stop_id,stop_name,stop_lat,stop_lon,geometry
8501,gen:8425:34005:2:3,Beimerstetten,48.481900,9.977156,POINT (9.97716 48.48190)
8502,gen:8425:34186:0:3,Lonsee Hauptstr.,48.543721,9.917794,POINT (9.91779 48.54372)
8503,gen:8425:34187:0:3,Urspring Abzw. Lonsee,48.546823,9.893904,POINT (9.89390 48.54682)


In [10]:
gdf_vvs_area = gpd.read_file(fileString_vvs_area_geom)

In [11]:
gdf_vvs_area.head(3)

,ags,gen,bez,geometry
0,08416036,Rottenburg am Neckar,Stadt,"POLYGON ((8.83443 48.48249, 8.80690 48.47901, ..."
1,08111000,Stuttgart,Stadt,"POLYGON ((9.22518 48.86601, 9.22500 48.86485, ..."
2,08115003,Böblingen,Stadt,"POLYGON ((9.05337 48.70321, 9.05941 48.70140, ..."


## Step 2: Filter gtfs data

In [12]:
gdf_gtfs.shape[0]

8504

In [13]:
gdf = gpd.sjoin(gdf_gtfs, gdf_vvs_area, how="left", op='intersects')

In [14]:
gdf.dropna(subset=['ags'], inplace=True)

In [15]:
gdf['stop_id'] = gdf['stop_id'].apply(lambda x: x.replace('gen:8', 'de:08'))

In [16]:
gdf.shape[0]

7673

In [17]:
gdf['ags_und_stop_name'] = gdf['ags'] + " " + gdf['stop_name']

In [18]:
gdf.head(3)

,stop_id,stop_name,stop_lat,stop_lon,geometry,index_right,ags,gen,bez,ags_und_stop_name
0,de:08111:100:1:1,Stammheim,48.850046,9.156174,POINT (9.15617 48.85005),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim
1,de:08111:100:2:3,Stammheim,48.849443,9.156022,POINT (9.15602 48.84944),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim
2,de:08111:100:2:4,Stammheim,48.850217,9.156093,POINT (9.15609 48.85022),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim


## Step 3: Do some some replacements

Manual replacements because Levenshtein Distance would bring missmatches...

In [19]:
to_replace = {'Jux': 'Spiegelberg',
              'Neuhausen (F)': 'Neuhausen auf den Fildern',
              'Gingen (F)': 'Gingen an der Fils', 
              'Rottenburg (N)': 'Rottenburg am Neckar', 
              'Dettingen (T)': 'Dettingen unter Teck',
              'Benningen (N)': 'Benningen am Neckar',
              'Liebersbronn': 'Liebersbr.',
             }

## Step 4: Match communities first

In [20]:
def lookUp(lookUpValue, df, lookUpcolumn, valueColumn):
    
    lst = df[df[lookUpcolumn] == lookUpValue][valueColumn].tolist()
    if len(lst)>0:
        return lst[0]
    else:
        return np.nan

In [21]:
def getLevenshteinDistance(string, key):
    
    return fuzz.ratio(string, key)
    

def findBestMatching(string, keys):
    
    distances = dict((key, getLevenshteinDistance(string, key)) for key in keys)
    distances = sorted((v,k) for k,v in distances.items())
    return distances[-1][1], distances[-1][0]

In [22]:
df_vz['gen_match'], df_vz['gen_sim'] = zip(*df_vz.apply(lambda x: findBestMatching(x['ort_1'], gdf_vvs_area['gen'].values), axis=1))
df_vz['ags_match'] = df_vz['gen_match'].apply(lambda x: lookUp(x, gdf_vvs_area, "gen", "ags"))
df_vz.head(3)

,ort_1,ort_2,haltestelle,nr,tarifzone,gen_match,gen_sim,ags_match
0,Affalterbach,NaN,Birkhau,3600,3,Affalterbach,100,08118001
1,Affalterbach,NaN,Klingenstraße,3601,3,Affalterbach,100,08118001
2,Affalterbach,NaN,Marbacher Straße,3599,3,Affalterbach,100,08118001


In [23]:
df_vz[df_vz['ags_match'].isna()]

,ort_1,ort_2,haltestelle,nr,tarifzone,gen_match,gen_sim,ags_match


In [24]:
def concatColumns(x):
    
    if (pd.isna(x.ort_2)):   
        return str(x.ags_match) + " " + str(x.ort_1) + " " + str(x.haltestelle)
    else:
        return str(x.ags_match) + " " + str(x.ort_2) + " " + str(x.haltestelle)

In [25]:
df_vz['ags_ort_und_haltestelle'] = df_vz.apply(lambda x: concatColumns(x), axis=1)
df_vz['ags_ort_und_haltestelle'] = df_vz['ags_ort_und_haltestelle'].str.replace(r'\b(\w+)(\s+\1)+\b', r'\1')

In [26]:
df_vz.head(3)

,ort_1,ort_2,haltestelle,nr,tarifzone,gen_match,gen_sim,ags_match,ags_ort_und_haltestelle
0,Affalterbach,NaN,Birkhau,3600,3,Affalterbach,100,08118001,08118001 Affalterbach Birkhau
1,Affalterbach,NaN,Klingenstraße,3601,3,Affalterbach,100,08118001,08118001 Affalterbach Klingenstraße
2,Affalterbach,NaN,Marbacher Straße,3599,3,Affalterbach,100,08118001,08118001 Affalterbach Marbacher Straße


## Step 4: Match full stop names afterwards

In [27]:
df_vz.head(3)

,ort_1,ort_2,haltestelle,nr,tarifzone,gen_match,gen_sim,ags_match,ags_ort_und_haltestelle
0,Affalterbach,NaN,Birkhau,3600,3,Affalterbach,100,08118001,08118001 Affalterbach Birkhau
1,Affalterbach,NaN,Klingenstraße,3601,3,Affalterbach,100,08118001,08118001 Affalterbach Klingenstraße
2,Affalterbach,NaN,Marbacher Straße,3599,3,Affalterbach,100,08118001,08118001 Affalterbach Marbacher Straße


In [28]:
gdf['best_match'], gdf['similiarity'] = zip(*gdf.apply(lambda x: findBestMatching(x['ags_und_stop_name'], df_vz['ags_ort_und_haltestelle'].values), axis=1))

In [29]:
gdf['zone'] = gdf['best_match'].apply(lambda x: lookUp(x, df_vz, "ags_ort_und_haltestelle", "tarifzone"))

In [30]:
gdf.head(3)

,stop_id,stop_name,stop_lat,stop_lon,geometry,index_right,ags,gen,bez,ags_und_stop_name,best_match,similiarity,zone
0,de:08111:100:1:1,Stammheim,48.850046,9.156174,POINT (9.15617 48.85005),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,08111000 Stuttgart Stammheim,78,1/2
1,de:08111:100:2:3,Stammheim,48.849443,9.156022,POINT (9.15602 48.84944),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,08111000 Stuttgart Stammheim,78,1/2
2,de:08111:100:2:4,Stammheim,48.850217,9.156093,POINT (9.15609 48.85022),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,08111000 Stuttgart Stammheim,78,1/2


## Step 5: Detect outliers by distance to Stuttgart Hbf

**Find outliers based on distance to Stuttgart Hauptbahnhof**

In [31]:
stg_hbf = (9.182316, 48.783332)
stg_hbf

(9.182316, 48.783332)

In [32]:
gdf['dist_stg_hbf'] = gdf.apply(lambda x: distance(stg_hbf, (x['stop_lon'], x['stop_lat'])).km, axis=1)

In [33]:
outliers = gdf.groupby('zone').agg(percentile_25 = ('dist_stg_hbf',lambda x: x.quantile(0.25)), 
                                   iqr = ('dist_stg_hbf',lambda x: x.quantile(0.75) - x.quantile(0.25)),
                                   percentile_75 = ('dist_stg_hbf',lambda x: x.quantile(0.75))
                                  ).reset_index(drop=False)

outliers['lb'] = outliers['percentile_25'] - 1.5*outliers['iqr']
outliers['ub'] = outliers['percentile_75'] + 1.5*outliers['iqr']

outliers

,zone,percentile_25,iqr,percentile_75,lb,ub
0,1,3.001994,3.937339,6.939333,-2.904015,12.845342
1,1/2,8.321821,2.473035,10.794857,4.612268,14.504410
2,2,11.773419,3.552208,15.325627,6.445106,20.653939
3,2/3,14.565975,4.159526,18.725501,8.326685,24.964790
4,3,19.830447,4.332225,24.162672,13.332109,30.661009
5,3/4,19.799245,11.024929,30.824174,3.261852,47.361567
6,4,25.178002,10.157509,35.335510,9.941739,50.571773
7,4/5,23.621530,8.696337,32.317867,10.577024,45.362373
8,5,34.256994,13.451881,47.708875,14.079172,67.886697
9,5/6,35.143061,20.474352,55.617413,4.431533,86.328941


In [34]:
gdf['out_lb'] = gdf.apply(lambda x: lookUp(x['zone'], outliers, 'zone', 'lb'), axis=1)
gdf['out_ub'] = gdf.apply(lambda x: lookUp(x['zone'], outliers, 'zone', 'ub'), axis=1)

In [35]:
gdf['out_by_distance'] = ((gdf['dist_stg_hbf'] < gdf['out_lb']) | (gdf['dist_stg_hbf'] > gdf['out_ub']))
gdf['out_by_sim_95'] = (gdf['similiarity']<95)
gdf['out_by_sim_85'] = (gdf['similiarity']<85)
gdf['out_by_sim_75'] = (gdf['similiarity']<75)

In [36]:
gdf.head(3)

,stop_id,stop_name,stop_lat,stop_lon,geometry,index_right,ags,gen,bez,ags_und_stop_name,best_match,similiarity,zone,dist_stg_hbf,out_lb,out_ub,out_by_distance,out_by_sim_95,out_by_sim_85,out_by_sim_75
0,de:08111:100:1:1,Stammheim,48.850046,9.156174,POINT (9.15617 48.85005),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,08111000 Stuttgart Stammheim,78,1/2,7.881738,4.612268,14.50441,False,True,True,False
1,de:08111:100:2:3,Stammheim,48.849443,9.156022,POINT (9.15602 48.84944),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,08111000 Stuttgart Stammheim,78,1/2,7.826419,4.612268,14.50441,False,True,True,False
2,de:08111:100:2:4,Stammheim,48.850217,9.156093,POINT (9.15609 48.85022),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,08111000 Stuttgart Stammheim,78,1/2,7.902514,4.612268,14.50441,False,True,True,False


## Step 6: Detect outliers by ags group

In [37]:
stopsPerAgs = gdf.groupby('ags').agg(stops = ('stop_id',lambda x: len(x))
                                    ).reset_index(drop=False)

stopsPerAgsAndZone = gdf.groupby(['ags', 'zone']).agg(stops = ('stop_id',lambda x: len(x)),
                                                     ).reset_index(drop=False)

idx = stopsPerAgsAndZone.groupby(['ags'])['stops'].transform(max) == stopsPerAgsAndZone['stops']

stopsPerAgsAndZone = stopsPerAgsAndZone[idx].reset_index(drop=True)

In [38]:
gdf['zone_by_ags'] = gdf.apply(lambda x: lookUp(x['ags'], stopsPerAgsAndZone, 'ags', 'zone'), axis=1)
gdf['zone_not_ags_zone'] = gdf['zone_by_ags'] != gdf['zone']

gdf.head(3)

,stop_id,stop_name,stop_lat,stop_lon,geometry,index_right,ags,gen,bez,ags_und_stop_name,...,zone,dist_stg_hbf,out_lb,out_ub,out_by_distance,out_by_sim_95,out_by_sim_85,out_by_sim_75,zone_by_ags,zone_not_ags_zone
0,de:08111:100:1:1,Stammheim,48.850046,9.156174,POINT (9.15617 48.85005),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,...,1/2,7.881738,4.612268,14.50441,False,True,True,False,1,True
1,de:08111:100:2:3,Stammheim,48.849443,9.156022,POINT (9.15602 48.84944),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,...,1/2,7.826419,4.612268,14.50441,False,True,True,False,1,True
2,de:08111:100:2:4,Stammheim,48.850217,9.156093,POINT (9.15609 48.85022),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,...,1/2,7.902514,4.612268,14.50441,False,True,True,False,1,True


In [39]:
gdf_vvs_area['zone'] = gdf_vvs_area.apply(lambda x: lookUp(x['ags'], stopsPerAgsAndZone, 'ags', 'zone'), axis=1)

## Step 7: Correct outliers

In [40]:
def correct_outlier(x):
    
    if x['ags'] == '08111000':
        
        # Do not correct Stuttgart zone
        return x['ini_zone']
        
    else:
        
        if ((not x['out_by_sim_75']) & (not x['out_by_distance']) & (not x['zone_not_ags_zone'])):
            
            return x['ini_zone']
        
        elif (x['out_by_sim_75'] & (not x['zone_not_ags_zone'])):
                
            return x['zone_by_ags']
        
        elif (not x['out_by_sim_85']):
            
            return x['ini_zone']
        
        else:
            
            return np.nan

In [41]:
gdf['ini_zone'] = gdf['zone']
gdf['zone'] = gdf.apply(lambda x: correct_outlier(x), axis=1)

In [42]:
gdf.head(10)

,stop_id,stop_name,stop_lat,stop_lon,geometry,index_right,ags,gen,bez,ags_und_stop_name,...,dist_stg_hbf,out_lb,out_ub,out_by_distance,out_by_sim_95,out_by_sim_85,out_by_sim_75,zone_by_ags,zone_not_ags_zone,ini_zone
0,de:08111:100:1:1,Stammheim,48.850046,9.156174,POINT (9.15617 48.85005),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,...,7.881738,4.612268,14.50441,False,True,True,False,1,True,1/2
1,de:08111:100:2:3,Stammheim,48.849443,9.156022,POINT (9.15602 48.84944),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,...,7.826419,4.612268,14.50441,False,True,True,False,1,True,1/2
2,de:08111:100:2:4,Stammheim,48.850217,9.156093,POINT (9.15609 48.85022),1.0,08111000,Stuttgart,Stadt,08111000 Stammheim,...,7.902514,4.612268,14.50441,False,True,True,False,1,True,1/2
3,de:08111:102:1:1,Korntaler Straße,48.846735,9.157090,POINT (9.15709 48.84674),1.0,08111000,Stuttgart,Stadt,08111000 Korntaler Straße,...,7.506192,4.612268,14.50441,False,True,True,False,1,True,1/2
4,de:08111:102:2:2,Korntaler Straße,48.846744,9.157159,POINT (9.15716 48.84674),1.0,08111000,Stuttgart,Stadt,08111000 Korntaler Straße,...,7.504301,4.612268,14.50441,False,True,True,False,1,True,1/2
5,de:08111:102:3:2,Korntaler Straße,48.846042,9.157497,POINT (9.15750 48.84604),1.0,08111000,Stuttgart,Stadt,08111000 Korntaler Straße,...,7.418782,4.612268,14.50441,False,True,True,False,1,True,1/2
6,de:08111:102:3:3,Korntaler Straße,48.845835,9.157592,POINT (9.15759 48.84584),1.0,08111000,Stuttgart,Stadt,08111000 Korntaler Straße,...,7.393774,4.612268,14.50441,False,True,True,False,1,True,1/2
7,de:08111:103:0:1,Heutingsheimer Straße,48.842443,9.159298,POINT (9.15930 48.84244),1.0,08111000,Stuttgart,Stadt,08111000 Heutingsheimer Straße,...,6.977667,4.612268,14.50441,False,True,False,False,1,True,1/2
8,de:08111:103:0:2,Heutingsheimer Straße,48.842452,9.159338,POINT (9.15934 48.84245),1.0,08111000,Stuttgart,Stadt,08111000 Heutingsheimer Straße,...,6.976932,4.612268,14.50441,False,True,False,False,1,True,1/2
9,de:08111:103:0:4,Heutingsheimer Straße,48.842623,9.159353,POINT (9.15935 48.84262),1.0,08111000,Stuttgart,Stadt,08111000 Heutingsheimer Straße,...,6.993852,4.612268,14.50441,False,True,False,False,1,True,1/2


In [43]:
gdf.columns

Index(['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'geometry',
       'index_right', 'ags', 'gen', 'bez', 'ags_und_stop_name', 'best_match',
       'similiarity', 'zone', 'dist_stg_hbf', 'out_lb', 'out_ub',
       'out_by_distance', 'out_by_sim_95', 'out_by_sim_85', 'out_by_sim_75',
       'zone_by_ags', 'zone_not_ags_zone', 'ini_zone'],
      dtype='object')

## Step 7: Export

In [44]:
stops_out = gdf.copy()
stops_out = stops_out[['stop_id', 'stop_name', 'ags', 'ags_und_stop_name', 'best_match', 'similiarity' ,'zone', 'ini_zone', 'geometry', 'out_by_distance', 'out_by_sim_95', 'out_by_sim_85', 'out_by_sim_75', 'zone_not_ags_zone']]
stops_out.to_csv("C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/01_prep/03_FareZones/stops_out.csv")

In [45]:
gdf_vvs_area.to_csv("C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/01_prep/03_FareZones/ags_out.csv")